# **Notebook 05**

Todos os scripts usam a sintaxe T-SQL.

Os resultados da consultas foram limitados a um número específico de linhas, para facilitar a visualização do notebook

  

* * *

## **Desafio**

### Com uma única query, retorne o volume total de pedidos, o volume de pedidos entregues, o volume de pedidos entregues dentro do prazo previsto e o volume de pedidos que não foram entregues no prazo.

- ### Agrupe os resultados por ano e mês.
- ### Ordene os resultado por ano e mês (em formato numérico) ascendente, volume total descendente e volume dentro do prazo ascendente.
- ### Filtre os meses que têm um volume de entrega fora do prazo menor do que 100.

<br>
A consulta abaixo oferece uma maneira mais direta de se obter o resultado solicitado:

In [17]:
SELECT 
     YEAR(order_purchase_timestamp) AS ano
    ,MONTH(order_purchase_timestamp) AS mes
    ,COUNT(order_id) AS total_pedidos
    ,COUNT(order_delivered_customer_date) AS total_pedidos_entregues
    ,SUM(IIF(order_delivered_customer_date <= order_estimated_delivery_date, 1, 0)) AS pedidos_entregues_no_prazo
    ,SUM(IIF(order_delivered_customer_date > order_estimated_delivery_date, 1, 0)) AS pedidos_entregues_fora_do_prazo
    ,IIF(
        (COUNT(order_delivered_customer_date) = 
           SUM(IIF(order_delivered_customer_date <= order_estimated_delivery_date, 1, 0)) + SUM(IIF(order_delivered_customer_date > order_estimated_delivery_date, 1, 0))
        ), CAST(1 AS BIT), CAST(0 AS BIT)) AS total_entregas_confere
FROM olist_orders
GROUP BY YEAR(order_purchase_timestamp), MONTH(order_purchase_timestamp)
HAVING SUM(IIF(order_delivered_customer_date > order_estimated_delivery_date, 1, 0)) < 100
ORDER BY ano ASC, mes ASC, total_pedidos DESC, pedidos_entregues_no_prazo ASC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(8 rows affected)

Total execution time: 00:00:01.045

ano,mes,total_pedidos,total_pedidos_entregues,pedidos_entregues_no_prazo,pedidos_entregues_fora_do_prazo,total_entregas_confere
2016,9,4,1,0,1,1
2016,10,317,270,267,3,1
2016,12,1,1,1,0,1
2017,1,790,750,727,23,1
2017,2,1735,1653,1600,53,1
2018,6,6167,6096,6013,83,1
2018,9,16,0,0,0,1
2018,10,4,0,0,0,1


<br>
Embora mais concisa, a query acima mistura funções e agregações nas cláusulas GROUP BY (linha 13) e HAVING (linha 14) em vez de se referirem aos alias das colunas correspondentes `ano`, `mes` e `pedidos_entregues_fora_do_prazo` - pois a ordem de execução da implementação do SQL exige que GROUP BY e HAVING sejam executados antes do SELECT, onde os alias foram definidos. Neste sentido, poderíamos melhorar um pouco a legibilidade refatorando a consulta utilizando subqueries:

In [22]:
SELECT 
     c.ano
    ,c.mes 
    ,COUNT(c.order_id) AS total_pedidos
    ,COUNT(c.order_delivered_customer_date) AS total_pedidos_entregues
    ,SUM(c.pedido_entregue_no_prazo) AS pedidos_entregues_no_prazo
    ,SUM(c.pedido_entregue_fora_do_prazo) AS pedidos_entregues_fora_do_prazo
    ,IIF(
            (COUNT(c.order_delivered_customer_date) = SUM(c.pedido_entregue_no_prazo) + SUM(c.pedido_entregue_fora_do_prazo)), 
            CAST(1 AS BIT), CAST(0 AS BIT)) AS total_entregas_confere
FROM (
    SELECT 
        order_id
        ,order_delivered_customer_date
        ,YEAR(order_purchase_timestamp) AS ano
        ,MONTH(order_purchase_timestamp) AS mes
        ,IIF(order_delivered_customer_date <= order_estimated_delivery_date, 1, 0) AS pedido_entregue_no_prazo
        ,IIF(order_delivered_customer_date > order_estimated_delivery_date, 1, 0) AS pedido_entregue_fora_do_prazo
    FROM olist_orders
) AS c
GROUP BY c.ano, c.mes
HAVING SUM(c.pedido_entregue_fora_do_prazo) < 100
ORDER BY ano ASC, mes ASC, total_pedidos DESC, pedidos_entregues_no_prazo ASC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(8 rows affected)

Total execution time: 00:00:00.527

ano,mes,total_pedidos,total_pedidos_entregues,pedidos_entregues_no_prazo,pedidos_entregues_fora_do_prazo,total_entregas_confere
2016,9,4,1,0,1,1
2016,10,317,270,267,3,1
2016,12,1,1,1,0,1
2017,1,790,750,727,23,1
2017,2,1735,1653,1600,53,1
2018,6,6167,6096,6013,83,1
2018,9,16,0,0,0,1
2018,10,4,0,0,0,1


<br>
Uma outra forma de se chegar ao mesmo resultado seria com a utilização de Common Table Expressions (CTEs):

In [23]:
WITH cte_desafio (ano, mes, order_id, order_delivered_customer_date, order_estimated_delivery_date, pedido_entregue_no_prazo, pedido_entregue_fora_do_prazo)
AS (
    SELECT 
         YEAR(order_purchase_timestamp) AS ano
        ,MONTH(order_purchase_timestamp) AS mes
        ,order_id
        ,order_delivered_customer_date
        ,order_estimated_delivery_date
        ,IIF(order_delivered_customer_date <= order_estimated_delivery_date, 1, 0) AS pedido_entregue_no_prazo
        ,IIF(order_delivered_customer_date > order_estimated_delivery_date, 1, 0) AS pedido_entregue_fora_do_prazo
    FROM olist_orders
)
SELECT 
    ano
    ,mes 
    ,COUNT(order_id) AS total_pedidos
    ,COUNT(order_delivered_customer_date) AS total_pedidos_entregues
    ,SUM(pedido_entregue_no_prazo) AS pedidos_entregues_no_prazo
    ,SUM(pedido_entregue_fora_do_prazo) AS pedidos_entregues_fora_do_prazo
    ,IIF(
        (COUNT(order_delivered_customer_date) = 
           SUM(pedido_entregue_no_prazo) + SUM(pedido_entregue_fora_do_prazo)
        ), CAST(1 AS BIT), CAST(0 AS BIT)) AS total_entregas_confere
FROM cte_desafio
GROUP BY ano, mes
HAVING SUM(pedido_entregue_fora_do_prazo) < 100
ORDER BY ano ASC, mes ASC, total_pedidos DESC, pedidos_entregues_no_prazo ASC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(8 rows affected)

Total execution time: 00:00:00.268

ano,mes,total_pedidos,total_pedidos_entregues,pedidos_entregues_no_prazo,pedidos_entregues_fora_do_prazo,total_entregas_confere
2016,9,4,1,0,1,1
2016,10,317,270,267,3,1
2016,12,1,1,1,0,1
2017,1,790,750,727,23,1
2017,2,1735,1653,1600,53,1
2018,6,6167,6096,6013,83,1
2018,9,16,0,0,0,1
2018,10,4,0,0,0,1


<br>
<hr>
<br>

## **Exercícios**

1. **Crie uma query que retorne os valores distintos de cidade existentes na tabela de clientes:**

In [14]:
-- Obs: a query a seguir 
-- (1) retorna as colunas listadas no SELECT, 
-- (2) ordena pelo critério do ORDER BY, 
-- (3) remove as duplicatas apontadas pelo DISTINCT e 
-- (4) limita o resultado à quantidade indicada pelo operador TOP
SELECT 
    DISTINCT TOP(20)    UPPER(customer_city) AS cidade, 
                        customer_state AS estado
FROM dbo.olist_customers
ORDER BY cidade, estado;

(20 rows affected)

Total execution time: 00:00:01.171

cidade,estado
ABADIA DOS DOURADOS,MG
ABADIANIA,GO
ABAETE,MG
ABAETETUBA,PA
ABAIARA,CE
ABAIRA,BA
ABARE,BA
ABATIA,PR
ABDON BATISTA,SC
ABELARDO LUZ,SC
